In [23]:
import pickle as pickle
import os
import pandas as pd
import torch
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, Trainer, TrainingArguments, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer
import re


import sys
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
import os
import re
import time
from collections import Counter

import transformers
from transformers import BertTokenizer,AdamWeightDecay,TFRobertaModel,TFBertModel

import sklearn
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import StratifiedKFold

# import pororo

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [24]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install pororo

Hit:1 http://ppa.launchpad.net/webupd8team/java/ubuntu bionic InRelease        
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease   
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Get:6 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1484 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2262 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3098 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2660 kB]
Fetched 9756 kB in 4s (2625 kB/s)                        
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages will be upgraded:
  chromium-chromedriver
1 upgraded, 0 newly inst

In [25]:
# from load_data.py
class RE_Dataset(torch.utils.data.Dataset):
  """ Dataset 구성을 위한 class."""
  def __init__(self, pair_dataset, labels):
    self.pair_dataset = pair_dataset
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

def preprocessing_dataset(dataset):
  """ 처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
  subject_entity = []
  object_entity = []
  for i,j in zip(dataset['subject_entity'], dataset['object_entity']):
    i = i[1:-1].split(',')[0].split(':')[1]
    j = j[1:-1].split(',')[0].split(':')[1]

    subject_entity.append(i)
    object_entity.append(j)
  out_dataset = pd.DataFrame({'id':dataset['id'], 'sentence':dataset['sentence'],'subject_entity':subject_entity,'object_entity':object_entity,'label':dataset['label'],})
  return out_dataset

def load_data(dataset_dir):
  """ csv 파일을 경로에 맡게 불러 옵니다. """
  pd_dataset = pd.read_csv(dataset_dir)
  dataset = preprocessing_dataset(pd_dataset)
  
  return dataset

def tokenized_dataset(dataset, tokenizer):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
    temp = ''
    temp = e01 + '[SEP]' + e02
    concat_entity.append(temp)
  tokenized_sentences = tokenizer(
      concat_entity,
      list(dataset['sentence']),
      return_tensors="pt",
      padding=True, # "max_length"
      truncation=True,
      max_length=256, # ??
      add_special_tokens=True,
      )
  return tokenized_sentences

def label_to_num(label):
  num_label = []
  with open('../code/dict_label_to_num.pkl', 'rb') as f:
    dict_label_to_num = pickle.load(f)
  for v in label:
    num_label.append(dict_label_to_num[v])
  
  return num_label

In [26]:
train_dataset = load_data("../dataset/train/train.csv")
# dev_dataset = load_data("../dataset/train/dev.csv") # validation용 데이터는 따로 만드셔야 합니다.

train_label = label_to_num(train_dataset['label'].values)
# dev_label = label_to_num(dev_dataset['label'].values)

In [27]:
def chrome_setting():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)
  return driver

In [28]:
driver=chrome_setting()

In [29]:
# Crawling
def kor_to_trans(text_data, trans_lang,start_index,final_index):

    target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

    trans_list= []
    for i in tqdm(range(start_index,final_index)): 
        
        if (i!=0)&(i%99==0):
            time.sleep(2)
            print('{}th : '.format(i), backtrans)
            # np.save(data_path+'kor_to_eng_train_{}_{}.npy'.format(start_index,final_index),trans_list)
        
        try:
            driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
            time.sleep(1.5)
            element=WebDriverWait(driver, 10).until(target_present)
            time.sleep(0.1)
            backtrans = element.text 

            if (backtrans=='')|(backtrans==' '):
                element=WebDriverWait(driver, 10).until(target_present)
                backtrans = element.text 
                trans_list.append(backtrans)
            else:
                trans_list.append(backtrans)
        
        except:
            trans_list.append('')
            
    return trans_list

In [30]:
print(len(train_dataset))

32460


In [31]:
trans_list=[]
output = kor_to_trans(train_dataset['sentence'], 'ko', 0, len(train_dataset))
print(output[:10], len(output))

99th :  He is the fourth son of King Heonjong Seonghwaje, half brother of King Hyojong Hong Chije, and the birth father of King Sejong's family ritual.
198th :  Haenam-gun (gun governor Myeonghyeon-gwan) will provide Nutrition Plus to deliver supplements to pregnant women and infants.
297th :  In January 2017, the TWRP team launched an Android application that allows it to flash recovery using root access, but unlike recovery, the app is proprietary software.


In [10]:
# Reset selenium chrome driver
driver=chrome_setting()

In [21]:
def trans_to_kor(text_data, trans_lang,start_index,final_index):

    target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))
    trans_list = []
    for i in tqdm(range(start_index,final_index)): 
        
        if (i!=0)&(i%99==0):
            time.sleep(2)
            print('{}th : '.format(i), backtrans)
            #   np.save(data_path+'kr_title.npy',trans_list)
        
        try:
            driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
            time.sleep(1.5)
            element=WebDriverWait(driver, 10).until(target_present)
            time.sleep(0.5)
            backtrans = element.text 

            if (backtrans=='')|(backtrans==' '):
                element=WebDriverWait(driver, 10).until(target_present)
                backtrans = element.text 
                trans_list.append(backtrans)
            else:
                trans_list.append(backtrans)
            
        except:
            trans_list.append('')
        
    return trans_list

In [22]:
trans_list=[]
backtrans = trans_to_kor(output, 'en', 0, len(output))
print(backtrans)

99th :  그는 헌종 성화제의 넷째 아들이며, 효종 홍치제의 이복동생이며, 세종대왕의 가례의 생부이다.

['"Something"은 조지 해리슨이 썼고 비틀즈의 1969년 앨범 "Abbey Road"에 수록되었다.', '호남을 기반으로 한 바른정당과 대안신당, 민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 거듭난다.', '', '균일가 생활용품 전문점 아성다이소는 COVID-19 바이러스에 시달리고 있는 대구광역시에 행복상자를 전달했다고 10일 밝혔다.', '1967년 프로야구 드래프트 1순위로 요미우리 자이언츠에 입단했을 때 등번호는 8번이었다.', ': 유엔, 유럽의회, 북대서양조약기구(NATO), 국제이주기구, 세계보건기구(WHO), 지중해연합, 이슬람협력기구, 유럽안보협력기구, 국제통화기금, 세계무역기구. 프랑코포니도 있어요', '이에 따라 나폴리와의 계약을 연장한 마라도나는 1989년 팀을 UEFA컵 정상에 올려놓았고 이듬해 유럽 챔피언 AC 밀란을 꺾고 다시 한 번 세리에A 정상에 올랐다.', '박용오(朴龍五, 1937년 3월 19일 ~ 2009년 11월 4일)는 서울에서 태어나 두산그룹 회장, KBO 총재를 역임한 대한민국의 기업인이다.', '중국군에 대항할 수 없게 된 국민당은 수도를 타이베이로 옮기기로 결정하고 12월부터 남은 군대와 국유재산을 차례로 대만으로 이전하기 시작했다.', '특히 김동연 전 부총리를 비롯해 안규백 김종민 오제세 최운열 김정우 권칠승 맹성규 의원 등 민주당 소속 의원 8명이 영상 메시지를 보냈다.', '하비에르 파스토레는 아르헨티나 클럽 타에레스의 청소년 팀에서 축구를 시작했다.', '1990년대 중반 이후 태어난 세대를 대표하는 아이콘으로 알려진 미국의 싱어송라이터 빌리 아일리쉬(본명 빌리 오코넬19)가 팝의 새로운 역사를 썼다.', '2009년 9월, 미국 필라델피아 필리스의 야구 선수 박찬호는 MBC 스페셜에서 "박찬호는 당신을 잊지 않았다. "나는 최진실의 고통과 죽음을 이해한다. 나는 최진실의 사건에 

In [16]:
print(backtrans[6])

Accordingly, Maradona, who extended his contract with Napoli, led his team to the top of the UEFA Cup in 1989, and the following year, he won against European champion AC Milan and once again reached the top of Serie A.


In [ ]:
# per_place_of_death
# per_product
# per:siblings
# org:product
# per:other_family
# per:place_of_residence
# per:date_of_death
# per:religion
# per:parents
# org:dissolved
# per:place_of_birth
# 